*Getting Started with Tensorflow 2*

*Week 2 Lecture Notes*

*Coursera/Imperial College London*

-----------------

# Building a Sequential Model

### **Feedforward neural networks**


We import the `Sequential` class from `tensorflow.keras.models`. Here we're also importing the `Dense` layer type from `tensorflow.keras.layers.`







In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

The way we build our model is really simple. We just create an instance of the `Sequential` class. Here I'm calling the instance model. All we do is pass in a list of `keras` layers. So this model is a feed forward network with a single hidden layer.

When you create a `Dense` layer, you always have to say how many units should be in that layer. That's what we have in the first argument. You can also optionally pass in an activation. If you don't pass in anything for the activation, the `Dense` layer will have a linear activation or no activation by default. 


In [ ]:
model = Sequential([
  Dense(64, activation='relu'),
  Dense(10, activation='softmax')
])

We are not saying what size or shape the input is going to be. With this model definition, we don't yet have enough information to create all of the model's weights and biases. It's actually okay to do this and leave it until the training stage when you feed the data into the model before the weights and biases are created. Alternatively, you can explicitly tell the model what shape the input data will be at the build stage. Like here, I'm specifying that each input data example will be a one-dimensional vector of size 784. Now in this case, the weights and biases will be created and initialized straight away.

In [ ]:
model = Sequential([
  Dense(63, activation='relu', input_shape=(784,)),
  Dense(10, activation='softmax')
])

Here's an alternative way of building the exact same model. Instead of passing in a list of layers, you can use the model.add method to append additional layers to the model. So what we have on this slide is an equivalent model to what we had on the previous slide. Which method you choose is, I think largely just a matter of taste, but it might also depend on how you structure your code. For example, you might find it useful to add layers one at a time like I'm doing here, if you want for model building codes to depend on some conditions or loops or something like that.


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
 
model = Sequential()
 
model.add(Dense(64, activation='relu', input_shape=(784,)))
model.add(Dense(10, activation='softmax'))

Here's one last example before you put this into practice yourself. In this model, I'm importing two layer types; flatten and dense. You can see that I'm making the first layer, the flatten layer. Again, I'm giving the input shape to the first layer in my model so that the weights and biases are created when I define the model instance. Notice that the input shape here is two-dimensional and so that's why I'm flattening each data input to unroll it into a long one-dimensional vector of size 784 before sending it through to the first dense layer. 

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
 
model = Sequential([
  Flatten(input_shape=(28,28)), # (784, )
  Dense(64, activation='relu'),
  Dense(10, activation='softmax')
])

----------------

**Question:** How many trainable parameters does a feedforward network have with input shape (64,), three hidden layers with 16 units each and a final linear layer with 8 units?

**Answer:** 65x16 + 17x16 + 17x16 + 17x8 = 1720

-----------------

### **Convolutional neural networks**

The models we've built so far were all feed-forward networks. Which are also called Multilayer Perceptrons or MLPs. Let's see how to build convolutional neural networks by including convolutional and pooling layers in our model.

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Conv2D, MaxPooling2D

model = Sequential([
                    Conv2D(16,(3,3), activation='relu', input_shape=(32,32,3)),
                    MaxPooling2D((3,3)),
                    Flatten(),
                    Dense(64, activation='relu'),
                    Dense(10, activation='softmax')
])

The first layer is a convolutional layer. And these layers have two required arguments. The first is the number of filters. So, we have 16 filters in this layer. And the second is the shape of the convolutional kernel. So, let's take a look at our input shape. This input could well be an image with 32 by 32 pixels and 3 color channels. So, our kernel is convolving over the image with a window of 3 by 3 pixels using 16 filters.

The second layer is a MaxPooling layer. This layer has one required argument, which is the pooling window size. And that's also 3 by 3 pixels.

This is followed by a Flatten and layer. Which, remember, is unrolling each data example into a long one dimensional vector ready to be passed through to the final two Dense layers.


So, just to make sure we understand what's going on here. Let's take a look at the tensor shapes after each of these layers.

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Conv2D, MaxPooling2D

model = Sequential([
                    Conv2D(16,(3,3), activation='relu', input_shape=(32,32,3)),  # (None, 30, 30, 16)
                    MaxPooling2D((3,3)),                                         # (None, 10, 10, 16)
                    Flatten(),                                                   # (None, 1600)
                    Dense(64, activation='relu'),                                # (None, 64)
                    Dense(10, activation='softmax')                              # (None, 10)
])


The first dimension of every tensor has a value of none. And that's because the first dimension will always be the batch size. And that batch size is flexible. We can pass any number of examples in a batch to the model. Tensor flow represents this with the value none in the tensor shape. So, let's ignore the batch size for the moment and look at the remaining shape dimensions.

Remember, the input shape is 32 by 32 by 3. After being processed by the convolutional layer, this becomes a tensor of shape 30 by 30 by 16. The 16 is because we created the layer with 16 filters. The shape 30 by 30 is because these convolutional layers have no 0 padding by default. Sometimes you'll hear that referred to as valid padding. And also, the default stride is one. So, with a kernel of shape 3 by 3, that results in a 30 by 30 by 16 shaped tensor output.

Likewise, the pooling layer has a 3 by 3 window. And these are non-overlapping. So, that downsamples the input to a 10 by 10 by 16 shaped tensor.

The Flatten layer just unrolls this into a one dimensional vector, ignoring the batch size. So, 10 times 10 times 16 is 1,600. From here on, the shapes are easy to understand. This is just a Dense layer with 64 units. And the final Dense layer has 10 units.

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Conv2D, MaxPooling2D

model = Sequential([
                    Conv2D(16,kernel_size=(3,3), padding='SAME', 
                           activation='relu', input_shape=(32,32,3)),            # (None, 32, 32, 16)
                    MaxPooling2D(pool_size=(3,3)),                               # (None, 10, 10, 16)
                    Flatten(),                                                   # (None, 1600)
                    Dense(64, activation='relu'),                                # (None, 64)
                    Dense(10, activation='softmax')                              # (None, 10)
])

Here's how it would look like if we wanted SAME padding. That just makes sure that the spatial dimensions don't change. So, the 32 by 32 by 3 shaped input goes to a 32 by 32 by 16 output. And the rest of the shapes for the other layers of the same.

Often the window shapes of convolutional and pooling layers have the same size in each dimension. In that case, you can just write the kernel size or the pool size as a single integer. So here, this is the same as writing 3 by 3 explicitly, as we did before:

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Conv2D, MaxPooling2D

model = Sequential([
                    Conv2D(16,kernel_size=3, padding='SAME', 
                           activation='relu', input_shape=(32,32,3)),            # (None, 32, 32, 16)
                    MaxPooling2D(pool_size=3),                                   # (None, 10, 10, 16)
                    Flatten(),                                                   # (None, 1600)
                    Dense(64, activation='relu'),                                # (None, 64)
                    Dense(10, activation='softmax')                              # (None, 10)
])

---------------

**Question:** What will the output shape be of the following model (ignoring the batch size)?

`Sequential([Conv2D(64, (4, 4), activation='tanh', input_shape=(64, 64, 3))])`

**Answer:** `(61, 61, 64)`

----------------

**Question**: Why are pooling layers often included in convolutional neural networks? 

**Answer:** 

a. To allow a degree of translational invariance on the input

b. To downsample the spatial dimensions, thereby reducing the number of network parameters

-----------------





### **Compiling your model**

To start training our network on data, we're also going to need to define a loss function that will give us a measure of our model's performance and the optimization algorithm. 

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential([
                    Dense(64, activation='elu', input_shape=(32,)),
                    Dense(1, activation='sigmoid')
])

model.compile(
    optimizer='SGD',
    loss='binary_crossentropy'
)


We can also optionally define a set of metrics that we want to keep track of as the model is training. These metrics will be calculated for each epoch of training along with the evaluation of the loss function on the training data.

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential([
                    Dense(64, activation='elu', input_shape=(32,)),
                    Dense(1, activation='sigmoid')
])

model.compile(
    optimizer='sgd', # 'adam', 'rmsprop', 'adadelta'
    loss='binary_crossentropy', #'mean_squared_error', 'categorical_crossentropy'
    metrics=['accuracy', 'mae']
)

One thing that you'll be noticing is that Keras is giving us a really nice usable interface for setting up these models. And there are a lot of readable strings that we can pass in to many of the options in the Keras API like we see here in the activation functions and the options in the compile method.

It's definitely worth knowing though that each of these strings is a reference to another object or function and we can always use that object or function directly. For example, all the options I have here in the compile function could be given like this.

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential([
                    Dense(64, activation='elu', input_shape=(32,)),
                    Dense(1, activation='sigmoid')
])

model.compile(
    optimizer=tf.keras.optimizers.SGD(),
    loss=tf.keras.losses.BinaryCrossentropy(),
    metrics=[tf.keras.metrics.BinaryAccuracy(), tf.keras.metrics.MeanAbsoluteError()]
)

Instead of passing in the string `'SGD'`, I'm directly passing in an SGD object that comes from the `tf.keras.optimizers` module. Similarly for the loss, the `BinaryCrossentropy` loss is available in the `tf.keras.losses` module. And the metrics I'm using can be given like this, the `BinaryAccuracy` and the `MeanAbsoluteError` metrics are in the `tf.keras.metrics` module. The reason why you might want to do this is because it gives you greater flexibility as many of these objects themselves have options that you might want to have control over.

This next code cell shows you what this means:

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential([
                    Dense(64, activation='elu', input_shape=(32,)),
                    Dense(1, activation='linear')
])

model.compile(
    optimizer=tf.keras.optimizers.SGD(learning_rate=0.001, momentum=0.9, nesterov=True),
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.BinaryAccuracy(threshold=0.7), tf.keras.metrics.MeanAbsoluteError()]
)

Let's take a look now at the `BinaryCrossentropy` function. That also takes some options and here I'm setting the option `from_logits=True`. If you are watching closely, you might have also seen that I've changed the activation function in the last layer of the network from `sigmoid` to `linear`. In other words, now, there is no activation function and I could as well have left this argument out as the `linear` activation is the default.

And so the network is now outputting the logits which is any real value before it is squeezed through the sigmoid activation function. The from_logits=True option tells the model that it should take the output of the network. And the loss function itself should handle the squeezing of the output through the sigmoid activation.

Mathematically, there's no difference between this and what I had before but this way turns out to be a more numerically stable approach.

### **The fit method**

```
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential([
                    Dense(64, activation='elu', input_shape=(32,)),
                    Dense(100, activation='softmax')
])

model.compile(
    optimizer='rmsprop',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.fit(X_train, y_train)

# X_train: (num_samples, num_features)
# y_train: (num_samples, num_classes)

```


 Here, I'm assuming that the labels have been represented as a one-hot vector, so each row of `y_train` is a vector of length num_classes which is all zeros, except for a one in the place corresponding to the correct class. Or if all the labels have a sparse representation, so just a single integer for each label, then `y_train` could be a one-dimensional array with length equal to the number of samples. Notice that in this case, we should choose the `sparse_categorical_crossentropy` loss function. 


```
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential([
                    Dense(64, activation='elu', input_shape=(32,)),
                    Dense(100, activation='softmax')
])

model.compile(
    optimizer='rmsprop',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.fit(X_train, y_train)

# X_train: (num_samples, num_features)
# y_train: (num_samples,)

```

 Passing these arrays into model.fit, we will then train the model for one pass through the training data or one epoch. 

You'll probably want to train a model for more than just one epoch, though, and so here, I'm passing in the optional keyword argument, epochs equals 10. So the training will make 10 complete passes through the dataset.

By default, the batch size is set to 32 whenever you pass in the dataset inputs and outputs as single Numpy arrays as we're doing here. If you want to change that to use a different batch size, you can just use the batch size argument. Like here, I'm setting the model to train on minibatches of size 16.


```
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential([
                    Dense(64, activation='elu', input_shape=(32,)),
                    Dense(100, activation='softmax')
])

model.compile(
    optimizer='rmsprop',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.fit(X_train, y_train, epochs=10, batch_size=16)


```

Finally, an important point is that calling `model.fit` actually returns something, and it returns something called a TensorFlow history object. This object contains a record of the progress of the network during training in terms of the loss and the metrics that we defined when we compiled the model. This object is actually an example of something called a callback.


```
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential([
                    Dense(64, activation='elu', input_shape=(32,)),
                    Dense(100, activation='softmax')
])

model.compile(
    optimizer='rmsprop',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

history = model.fit(X_train, y_train, epochs=10, batch_size=16)

```

----------------

**Question:** Suppose we are building a classification model, where there are 1010 classes. The final output layer is given by `Dense(10, activation=‘softmax’)`. Our dataset labels are given by a single numpy array `y_train`. 

`y_train` has shape `(num_samples,)`, and the first two labels are: `y[0] = 9`, `y[1] = 0`. Which loss function should we use?

**Answer**: `sparse_categorical_crossentropy`

--------------

**Question:** Suppose instead that y_train has shape `(num_samples, 10)`, and the first two labels are 

`y[0] = [0, 0, 0, 0, 0, 0, 0, 0, 0, 1]`, 

`y[1] = [1, 0, 0, 0, 0, 0, 0, 0, 0, 0]`. 

Which loss function should we use?

**Answer:** `categorical_crossentropy`

--------------



### **Evaluation and prediction**

```
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential([Dense(1, activation='sigmoid', input_shape=(12,))])
model.compile(optimizer='sgd', loss='binary_crossentropy',metrics=['accuracy'])
model.fit(X_train, y_train)

loss, accuracy, mae = model.evaluate(X_test, y_test)

# X_sample: (num_samples, 12)

pred = model.predict(X_sample)  # [[0.07713523]]

```

The first dimension corresponds to the number of examples that we want to get predictions for and the rest of the dimensions will be the data features. In the example, I'm showing here, the number of features is 12 and notice how that corresponds to the input shape that I set when I defined the model. `model.predict` will then return the outputs of the network for these given inputs. 


One point that often catches people out, if we're getting predictions in only one example, then we still have to have a dummy first dimension that will be equal to one. So for example, if `X_sample` is a NumPy array, with just one input sample and our network is a binary classifier where the final layer has just one neuron passed through a sigmoid activation, then `model.predict` will return just one number that we interpreted as a probability that the input we passed in belongs to the positive class. 

```
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential([Dense(1, activation='sigmoid', input_shape=(12,))])
model.compile(optimizer='sgd', loss='binary_crossentropy',metrics=['accuracy'])
model.fit(X_train, y_train)

loss, accuracy, mae = model.evaluate(X_test, y_test)

# X_sample: (1, 12)

pred = model.predict(X_sample)  # [[0.07713523]]

```

See how the output in the `model.predict` is a two-dimensional NumPy array. The first dimension of this array, again, corresponds to the number of examples that we're getting predictions for. So if we were passing in two examples in a stack in NumPy array, and so example now is an array where the first dimension is equal to two, then we would get out a two-dimensional array where the first dimension is also equal to two. 

```
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
 
model = Sequential([Dense(1, activation='sigmoid', input_shape=(12,))])
model.compile(optimizer='sgd', loss='binary_crossentropy',metrics=['accuracy'])
model.fit(X_train, y_train)
 
loss, accuracy, mae = model.evaluate(X_test, y_test)
 
# X_sample: (2, 12)
 
pred = model.predict(X_sample)  # [[0.07713523]]
                                # [[0.94515101]]
 ```




As a final example, let's say, a network is a multi-class classification model where there are three classes. So our final layer has three neurons and softmax activation. You'll see I've also changed the loss-function from binary cross entropy to categorical cross entropy. I'm supposing that the number of input features is the same as before, 12. Let's also say that example again contains two data inputs. So that the first dimension of this array is equal to two. Then the output of `model.predict` will look something like this. The array of output predictions will be a two-dimensional array as before, the first dimension will be two, which matches the first dimension of X sample as the number of examples we're getting predictions for and the second dimension will be three for the number of classes. Each row of this array is the network output for that data input. The output layer is a softmax layer, remember, so each layer is a set of output probabilities emitted by the softmax function and you can see how they add up to one in each case. So here, it looks like the model is predicting class one for the first example and class three for the second. 


```
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
 
model = Sequential([Dense(3, activation='softmax', input_shape=(12,))])
model.compile(optimizer='sgd', loss='categorical_crossentropy',metrics=['accuracy'])
model.fit(X_train, y_train)
 
loss, accuracy, mae = model.evaluate(X_test, y_test)
 
# X_sample: (2, 12)
 
pred = model.predict(X_sample)  # [[0.93957397, 0.0189931, 0.04143293],
                                # [[0.01211542, 0.0907736, 0.89711098]]
 ```




-------------------------

**Question:** Suppose we have built, compiled and trained the following deep learning model for a regression task:

```
model = Sequential([
    Dense(64, input_shape=(16, 4)), 
    Flatten(), 
    Dense(2)
])
model.compile(optimizer=‘rmsprop’, loss=‘mean_squared_error’, metrics=[‘mae’, ‘mape’])
model.fit(X_train, y_train, epochs=10)

```
**a.** What should the shapes of  `X_test` and `y_test` be in the following model evaluation?

`model.evaluate(X_test, y_test)`


**Answer:** 
`(num_test_samples, 16, 4), (num_test_samples, 2)`

**b.** What should the shape of X_sample be in a call to `model.predict(X_sample)`?

**Answer:** `(num_samples, 16, 4)`






--------------------